This notebook demonstrates how to optimize the parameters of the state-space model (implemented in **ReactiveMP.jl**) through an external optimization packages such as [Optim.jl](https://github.com/JuliaNLSolvers/Optim.jl/)

We use the following model:
$$
\begin{align*}
    {x}_t &= {x}_{t-1} + c \\
    {y}_t &\sim \mathcal{N}\left({x}_{t}, p \right) 
\end{align*}
$$
with prior $${x}_0 \sim \mathcal{N}({m_{{x}_0}}, {v_{{x}_0}})$$.

Our goal is to optimize parameters $c$ and ${m_{{x}_0}}$.

In [7]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions

In [8]:
@model function smoothing(n, x0, c::ConstVariable, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)

    x_prev = x_prior

    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

In [9]:
using Random

In [20]:
P = 1.0

Random.seed!(123)

n = 250
c_real = -5.0
data = c_real .+ collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [21]:
# c[1] is C
# c[2] is μ0
function f(c)
    x0_prior = NormalMeanVariance(c[2], 100.0)
    result = inference(model=Model(smoothing, n, x0_prior, c[1], P), data=(y=data,), free_energy=true)
    return result.free_energy[end]
end

f (generic function with 1 method)

In [22]:
using Optim

In [23]:
res = optimize(f, ones(2), GradientDescent(), Optim.Options(g_tol = 1e-3, iterations = 100, store_trace = true, show_trace = true))

Iter     Function value   Gradient norm 
     0     3.655789e+02     8.149754e+02
 * time: 0.00013899803161621094
     1     3.653239e+02     5.997070e-02
 * time: 0.10898494720458984
     2     3.652297e+02     3.418191e+02
 * time: 0.7918410301208496
     3     3.651848e+02     2.856214e-02
 * time: 0.8754818439483643
     4     3.651653e+02     1.628199e+02
 * time: 1.5290229320526123
     5     3.651552e+02     1.491695e-02
 * time: 1.6358189582824707
     6     3.651483e+02     8.271286e+01
 * time: 2.3205809593200684
     7     3.651457e+02     5.697649e-03
 * time: 2.429054021835327
     8     3.651449e+02     3.250149e+01
 * time: 3.112091064453125
     9     3.651445e+02     2.921074e-03
 * time: 3.2149858474731445
    10     3.651444e+02     1.173193e+01
 * time: 3.8436949253082275
    11     3.651443e+02     2.498033e-03
 * time: 3.9480140209198
    12     3.651443e+02     9.145920e+00
 * time: 4.578362941741943
    13     3.651443e+02     2.207014e-03
 * time: 4.68285083770

 * Status: success

 * Candidate solution
    Final objective value:     3.651440e+02

 * Found with
    Algorithm:     Gradient Descent

 * Convergence measures
    |x - x'|               = 3.46e-06 ≰ 0.0e+00
    |x - x'|/|x'|          = 6.93e-07 ≰ 0.0e+00
    |f(x) - f(x')|         = 7.77e-06 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.13e-08 ≰ 0.0e+00
    |g(x)|                 = 1.37e-04 ≤ 1.0e-03

 * Work counters
    Seconds run:   8  (vs limit Inf)
    Iterations:    21
    f(x) calls:    158
    ∇f(x) calls:   158


In [24]:
res.minimizer # Real values are indeed (c = 1.0 and μ0 = -5.0)

2-element Vector{Float64}:
  1.0006316025049258
 -4.984289609532286

In [25]:
println("Real value vs Optimized")
println("Real:      ", [c, c_real])
println("Optimized: ", res.minimizer)

Real value vs Optimized
Real:      [1.0, -5.0]
Optimized: [1.0006316025049258, -4.984289609532286]
